# Notas del curso CS229 - Machine Learning

---

# Regresión lineal - LMS (Maximum Likelihood - Gradiente Descendiente)

El primer algoritmo de aprendizaje que muestran en el curso (y en bishop también) es el LMS. (En realidad, le ponen muchos nombres a la misma cosa, pero básicamente "el aprendizaje" es con LMS.) Entonces, la explicación es la que sigue.

## Interpretación determinística:

Primero, vamos a ver el problema desde un punto de vista determinístico. Se tiene un conjunto $\{(x^{(1)},y^{(1)}) \ldots,(x^{(m)},y^{(m)})\}$ tal que $x^{(i)} \in \mathbb{R}^n \;\forall i=1,\ldots,m$ son los vectores de features y $y^{(i)} \in \mathbb{R} \;\forall i=1,\ldots,m$ son las etiquetas de cada uno de ellos. 

Ahora, se decide obtener una función $h_{\theta}(x)$ que permita predecir nuevos valores de $y$ a partir de un nuevo $x$ dado. Es decir, ni $x$ ni $y$ pertenecen al conjunto anterior. Son nuevos vectores que van a aparecer cuando quiera hacer una predicción usando la función $h_{\theta}(x)$ encontrada.

Ahora, ¿cómo encontramos la función $h_{\theta}(x)$? Vamos a definir que la función tiene la forma:
$$
h_{\theta}(x) = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \ldots + \theta_n x_n = \theta^T x
$$
y que $\theta$ va a ser el vector que minimice la función
$$
J(\theta) = \frac{1}{2}\sum_{i=1}^{n}(y^{(i)} - h_{\theta}(x^{(i)}))
$$

Este problema es el conocido problema de minimización por cuadrados mínimos, el cual se minimiza de la siguiente manera... HAY QUE SEGUIR Y ME DA UN POQUITO DE FIACA.

También se puede minimizar haciendo gradiente descendiente:
$$
\theta^{(j+1)} = \theta^{(j)} - \alpha \nabla_\theta J(\theta^{(j)})
$$
lo cual queda el siguiente algoritmo:
$$
\theta^{(j+1)}_i = \theta^{(j)}_i - \alpha (h_{\theta^{(j)}}(x^{(i)}) - y^{(i)})x_j^{(i)}
$$
donde 
* $\theta^{(j)}_i$ es el valor de la componente $i$ del vector $\theta$ en la iteración $j$
* $\theta^{(j)}$ es todo el vector $\theta$ en la iteración $j$
* $x^{(i)}_j$ es el valor de la componente $j$ del vector $x^(i)$
* $y^{(i)}$ es la etiqueta de $x^{(i)}$

y este es algoritmo LMS.


## Interpretación probabilística:

Se puede llegar a la misma ecuación que antes desde una interpretación probabilística, con la ventaja de que ahora se puede ver como caso particular de un conjunto de algoritmos más grande y hay más herramientas para probar convergencia y esas cosas.

Se tiene una muestra de la vector aleatorio $(\mathbf{x},\mathbf{y})$, es decir un conjunto $\{(\mathbf{x}^{(1)},\mathbf{y}^{(1)}) \ldots,(\mathbf{x}^{(m)},\mathbf{y}^{(m)})\}$, donde ahora $\mathbf{x}^{(i)}$ y $\mathbf{y}^{(i)}$ son variables aleatorias iid (no entre sí) de las mismas dimensiones que antes.

Si suponemos que la variable aleatoria $\mathbf{y}$ se genera mediante la ecuación
$$
\mathbf{y} = h_\theta(\mathbf{x}) + \epsilon
$$
donde $\epsilon \sim \mathcal{N}(0,\sigma^2)$, entonces el likelihood de la muestra aleatoria se puede calcular de la siguiente manera:
$$
l(\theta) = P\left((\mathbf{x}^{(1)},\mathbf{y}^{(1)}) = (x^{(1)},y^{(1)}), \ldots, (\mathbf{x}^{(m)},\mathbf{y}^{(m)}) = (x^{(m)},y^{(m)})\right) = \prod_{i=1}^m P(x^{(i)},y^{(i)})
$$

NOTA SOBRE EL LIKELIHOOD: La función $l(\theta)$ es una función de $\theta$ y de la **realización** de la muestra de $(\mathbf{x},\mathbf{y})$, es decir el conjunto $\{(x^{(1)},y^{(1)}) \ldots,(x^{(m)},y^{(m)})\}$, no del conjunto $\{(\mathbf{x}^{(1)},\mathbf{y}^{(1)}) \ldots,(\mathbf{x}^{(m)},\mathbf{y}^{(m)})\}$. Esto implica que no sólo tengo un valor de likelihood para cada valor de theta, sino que también para cada valor de $(x^{(i)},y^{(i)})$ y que la maximización por ML sea una cosa *relativamente* segura (en el sentido de encontrar el verdadero máximo). Es decir, una vez que tengo el $\theta$ que maximiza el likelihood, voy a tener que esperar a que venga la muestra para encontrar el valor del likelihood, el cual va a depender, de nuevo, de la realización de la muestra.

Ahora, el valor de $\theta$ para el cual se hace máxima esa función es se puede encontrar directamente de esa función, pero se obtiene que el likelihood termina siendo inversamente proporcional al error cuadrático medio, con lo cual el algoritmo termina siendo el mismo y se puede implementar el mismo procedimiento de maximización iterativa que antes con gradiente descendiente. El despeje sería:
$$
\begin{align}
l(\theta) &= \prod_{i=1}^m P(x^{(i)},y^{(i)})\\
\log(l(\theta)) &=  \sum_{i=1}^m \log\left(P(x^{(i)},y^{(i)})\right)\\
&=  \sum_{i=1}^m \log\left(P(y^{(i)}|x^{(i)})\right) \log\left(P(x^{(i)})\right)\\
&=  \sum_{i=1}^m \left(-\log(\sqrt{2\pi}\sigma) - \frac{1}{2\sigma^2}(y^{(i)} - h_\theta(x^{(i)}))^2\right)\log\left(P(x^{(i)})\right)
\end{align}
$$
si se asume $P(x^{(i)})$ uniforme, entonces esa probabilidad es constante. Entonces, el único término a maxsimizar es el que tiene el error cuadrático medio. PREGUNTA: SI LAPROBABILIDAD A PRIORI NO ES CONSTANTE, EL ALGORITMO ES EL QUE TIENE LOS PESOS ? 


In [1]:
import numpy as np
%matplotlib notebook
from matplotlib import pyplot as plt

In [2]:
N = 10000

# x ~ Ber(p)
x = np.random.rand(N)
p = .5
x[x>p] = 1
x[x<=p] = 0




array([1., 0., 0., 1., 1., 1., 0., 0., 1., 1.])